In [1]:
import numpy as np
import pandas as pd
from numpy.random import randn
from IPython.display import clear_output

In [2]:
cars=pd.read_csv('Reynoldsburg Carvana.csv')

In [3]:
cars.head()

,YEAR,BRAND,MAKE,TRIM,PRICE,MILES,SHIP,PAGE,ENTRY
0,2017,HYUNDAI,SONATA,SE,"14,600","24,223 miles",Shipping: $300,1,1
1,2017,KIA,SPORTAGE,EX,"21,400","2,948 miles",Free Shipping,1,2
2,2019,CHRYSLER,PACIFICA,Limited,"34,400","24,928 miles",Shipping: $200,1,3
3,2017,HYUNDAI,TUCSON,SE,"15,600","34,179 miles",Shipping: $300,1,4
4,2017,NISSAN,ROGUE,SV,"16,600","32,046 miles",Shipping: $300,1,5


In [4]:
cars.loc[cars['YEAR']==2020]
#dang. 55k toyota sports car hidden on carvana page 449, only 1000 miles on it too.

,YEAR,BRAND,MAKE,TRIM,PRICE,MILES,SHIP,PAGE,ENTRY
8961,2020,TOYOTA,GR SUPRA,3,"54,900","1,061 miles",Free Shipping,449,2


## clean price column

In [5]:
price=[]
dummy=[]
for x in cars['PRICE']:
    y=x.replace(',','')
    price.append(int(y))
    dummy.append(int(1))

## clean miles column

In [6]:
miles=[]

for x in cars['MILES']:
    y=x.replace(' miles','')
    z=y.replace(',','')
    miles.append(int(z))

## clean shipping column

In [7]:
shipping=[]

for x in cars['SHIP']:
    if x =='Free Shipping':
        shipping.append(int(0))
    else:
        y=x.replace('Shipping: $','')
        shipping.append(int(y))

## replace columns with cleaned

In [8]:
del cars['MILES']
del cars['PRICE']
del cars['SHIP']



In [9]:
cars['PRICE']=price
cars['MILES']=miles
cars['SHIPPING']=shipping
cars['dummy']=dummy

## remove car brands with less than 100 entries and not going to use make or trim

In [10]:
lst=[]
for brand,x in cars['BRAND'].value_counts().iteritems():
    if x>=100:
        lst.append(brand)

cleaned_cars=cars[cars['BRAND'].isin(lst)]

del cleaned_cars['MAKE']
del cleaned_cars['TRIM']

#lst= ['CHEVROLET', 'FORD', 'HYUNDAI', 'TOYOTA', 'JEEP', 'VOLKSWAGEN', 'NISSAN', 
#'KIA', 'BUICK', 'DODGE', 'HONDA', 'GMC', 'SUBARU', 'INFINITI', 'MAZDA', 'CADILLAC', 'FIAT', 'LEXUS', 'BMW']

## create and fill categorical columns for each brand 

In [11]:
for x in lst:
    cleaned_cars[x]=0
    

C:\Users\16142\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
for index,x in cleaned_cars.iterrows():
     cleaned_cars[x[1]].loc[index]=int(1)
   

C:\Users\16142\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\16142\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyboardInterrupt: 

## Remove duplicate rows

noticed towards the end that carvana was repeating the same page of cars over and over, that would lead to oversampling, so those rows were removed. 

should have done this before ennumerating categorical columns, the deduping process cut out 20k!!!! rows!

For me to remove a row I had to have the columns match year, brand, price, milage, and shipping cost. So the possibility of the deleted row being real is slim to none. Carvana really ballons their stock... 

In [137]:
cleaned_cars.columns

Index(['YEAR', 'BRAND', 'PAGE', 'ENTRY', 'PRICE', 'MILES', 'SHIPPING', 'dummy',
       'CHEVROLET', 'FORD', 'HYUNDAI', 'TOYOTA', 'JEEP', 'VOLKSWAGEN',
       'NISSAN', 'KIA', 'BUICK', 'DODGE', 'HONDA', 'GMC', 'SUBARU', 'INFINITI',
       'MAZDA', 'CADILLAC', 'FIAT', 'LEXUS', 'BMW'],
      dtype='object')

In [178]:
dedup=cleaned_cars[~cleaned_cars.duplicated(['YEAR']+['BRAND']+['PRICE']+['MILES']+['SHIPPING'],keep='first')]

In [180]:
dedup.to_csv('Cleaned_Carvana.csv',index=False)

In [169]:
dedup.loc[dedup['YEAR']==2020]

,YEAR,BRAND,PAGE,ENTRY,PRICE,MILES,SHIPPING,dummy,CHEVROLET,FORD,...,DODGE,HONDA,GMC,SUBARU,INFINITI,MAZDA,CADILLAC,FIAT,LEXUS,BMW
8961,2020,TOYOTA,449,2,54900,1061,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
cars.shape

(33916, 10)

In [174]:
dedup.shape

(8610, 27)

In [176]:
#percent of total real entries
100*(8610/22916)

37.57200209460639

In [ ]:
lst=[]
for brand,x in cars['BRAND'].value_counts().iteritems():
    if x>=100:
        lst.append(brand)

cleaned_cars=cars[cars['BRAND'].isin(lst)]